### 1\. 📥 Script de Extração: `01_cvm_downloader.py`

### 📝 Descrição

Este script é a porta de entrada dos dados. Ele conecta-se ao portal de Dados Abertos da CVM e realiza o download massivo de arquivos históricos.

  * **Fonte:** `dados.cvm.gov.br`
  * **Tipos de Documento:**
      * **DFP:** Demonstrações Financeiras Padronizadas (Anual).
      * **ITR:** Informações Trimestrais (Trimestral).
  * **Período:** 2011 a 2025.

### 🔄 Fluxograma de Execução

```mermaid
graph LR
    A[Início] --> B{Loop: Tipo Documento<br>DFP / ITR}
    B --> C{Loop: Anos<br>2011-2025}
    C --> D{Arquivo ZIP<br>já existe?}
    D -- Sim --> E[Pular Download]
    D -- Não --> F[Baixar arquivo .ZIP]
    F --> G[Descompactar em<br>/unzipped]
    E --> G
    G --> H[Próximo Ano]
    H --> I[Fim do Loop]
```

### 📂 Dados de Saída (Brutos)

O script gera uma estrutura de pastas contendo arquivos `.csv` brutos extraídos diretamente da CVM.

| Arquivo Exemplo | Descrição |
| :--- | :--- |
| `dfp_cia_aberta_2023.zip` | Arquivo compactado original. |
| `*_DRE_con_2023.csv` | CSV bruto da Demonstração de Resultado (Consolidado). |
| `*_BPA_con_2023.csv` | CSV bruto do Balanço Patrimonial Ativo (Consolidado). |

-----

In [ ]:
import datetime
import logging
import os
import time
from zipfile import ZipFile

import pandas as pd

from typing import Dict, Iterable, List, Optional, Set

import requests
import tqdm


BASE_DIR = os.path.join("..", "data", "cvm")
ZIP_DIR = os.path.join(BASE_DIR, "zip")
UNZIPPED_DIR = os.path.join(BASE_DIR, "unzipped")

os.makedirs(ZIP_DIR, exist_ok=True)
os.makedirs(UNZIPPED_DIR, exist_ok=True)

URL_BASE = "https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/{doc_type}/DADOS/"

DOC_TYPES = ['DFP', 'ITR']

YEARS = range(2011, 2026) 

def download_and_unzip(url, zip_path, unzipped_path):
    """Baixa e descompacta um arquivo ZIP se ele não existir localmente."""
    if os.path.exists(zip_path):
        print(f"Arquivo já existe, pulando download: {os.path.basename(zip_path)}")
    else:
        print(f"Baixando: {os.path.basename(zip_path)}")
        try:
            response = requests.get(url, stream=True)
            response.raise_for_status()
            
            total_size = int(response.headers.get('content-length', 0))
            
            with open(zip_path, 'wb') as f, tqdm.tqdm(
                desc=os.path.basename(zip_path),
                total=total_size,
                unit='iB',
                unit_scale=True,
                unit_divisor=1024,
            ) as bar:
                for data in response.iter_content(chunk_size=1024):
                    size = f.write(data)
                    bar.update(size)
            print("Download completo.")
        except requests.exceptions.RequestException as e:
            print(f"Erro no download de {url}: {e}")
            return 
    print(f"Descompactando: {os.path.basename(zip_path)}")
    try:
        with ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(unzipped_path)
        print("Descompactado com sucesso.")
    except Exception as e:
        print(f"Erro ao descompactar {zip_path}: {e}")

if __name__ == "__main__":
    for doc_type in DOC_TYPES:
        for year in YEARS:
            filename = f"{doc_type}_cia_aberta_{year}.zip"
            url = URL_BASE.format(doc_type=doc_type) + filename
            
            zip_path = os.path.join(ZIP_DIR, filename)
            unzipped_path = os.path.join(UNZIPPED_DIR, f"{doc_type}_{year}")
            
            download_and_unzip(url, zip_path, unzipped_path)
            print("-" * 50)
            
    print("\nProcesso de download e extração concluído!")

Baixando: DFP_cia_aberta_2011.zip


DFP_cia_aberta_2011.zip: 100%|██████████| 8.80M/8.80M [00:00<00:00, 10.2MiB/s]


Download completo.
Descompactando: DFP_cia_aberta_2011.zip
Descompactado com sucesso.
--------------------------------------------------
Baixando: DFP_cia_aberta_2012.zip


DFP_cia_aberta_2012.zip: 100%|██████████| 8.78M/8.78M [00:00<00:00, 9.69MiB/s]


Download completo.
Descompactando: DFP_cia_aberta_2012.zip
Descompactado com sucesso.
--------------------------------------------------
Baixando: DFP_cia_aberta_2013.zip


...

Download completo.
Descompactando: ITR_cia_aberta_2023.zip
Descompactado com sucesso.
--------------------------------------------------
Baixando: ITR_cia_aberta_2024.zip


ITR_cia_aberta_2024.zip: 100%|██████████| 31.2M/31.2M [00:02<00:00, 11.8MiB/s]


Download completo.
Descompactando: ITR_cia_aberta_2024.zip
Descompactado com sucesso.
--------------------------------------------------
Baixando: ITR_cia_aberta_2025.zip


ITR_cia_aberta_2025.zip: 100%|██████████| 29.2M/29.2M [00:02<00:00, 11.3MiB/s]


Download completo.
Descompactando: ITR_cia_aberta_2025.zip
Descompactado com sucesso.
--------------------------------------------------

Processo de download e extração concluído!


### 2\. 🧹 Script de Parsing: `02_cvm_parser.py`

### 📝 Descrição

Este script atua como um "filtro grosso". Ele varre os milhares de arquivos CSV descompactados, lê em pedaços (chunks), aplica regras de negócio para limpar "lixo" e consolida tudo em arquivos Parquet intermediários.

  * **Filtros Aplicados:**
      * Apenas demonstrativos **Consolidados** (`_con_`).
      * Apenas ordens de exercício **ÚLTIMO** ou **PENÚLTIMO**.
      * Correção de escala de moeda (`MIL` -\> `UNIDADE`).
      * Conversão de tipos de dados para otimização de memória.

### 🔄 Fluxograma de Execução

```mermaid
graph LR
    A[Início] --> B[Buscar CSVs Consolidados]
    B --> C[Ler CSV com Encoding Latin-1]
    C --> D{Filtros de Qualidade}
    D -- Dados Inválidos --> E[Descartar]
    D -- Dados Válidos --> F[Ajustar Escala Moeda<br>* 1000 se necessário]
    F --> G[Concatenar Chunks]
    G --> H[Remover Duplicatas]
    H --> I[Salvar RAW Parquet]
```

### 📊 Dataset Intermediário (`raw_*.parquet`)

Este script gera os arquivos `raw_dre.parquet`, `raw_bpa.parquet`, `raw_bpp.parquet`. A estrutura destes dados ainda é no formato **LONG** (uma linha por conta contábil).

| Coluna | Tipo | Descrição |
| :--- | :--- | :--- |
| `CNPJ_CIA` | `string` | Identificador único da empresa. |
| `DT_FIM_EXERC` | `datetime` | Data de referência do balanço. |
| `CD_CONTA` | `string` | Código contábil (ex: `3.01`). |
| `DS_CONTA` | `string` | Descrição da conta (ex: `Receita`). |
| `VL_CONTA` | `float` | Valor monetário da conta. |
| `ESCALA_MOEDA` | `category` | Escala original (MIL/UNIDADE). |

-----


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import logging
import gc

BASE_DIR = Path("..") / "data" / "cvm"
UNZIPPED_DIR = BASE_DIR / "unzipped"
PROCESSED_DIR = BASE_DIR / "processed"
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

DOC_PATTERNS_BROAD = {
    "dre": "*_DRE_con_*.csv",
    "bpa": "*_BPA_con_*.csv",
    "bpp": "*_BPP_con_*.csv",
}

CNPJ_SAPR = "76.484.013/0001-45"
CNPJs_DEBUG = [CNPJ_SAPR, "33.839.910/0001-11"] 

CATEGORY_COLS = [
    "CNPJ_CIA", "DENOM_CIA", "GRUPO_DFP", "MOEDA",
    "ESCALA_MOEDA", "ORDEM_EXERC", "CD_CONTA",
    "DS_CONTA", "ST_CONTA_FIXA"
]


logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s %(message)s")
logger = logging.getLogger("cvm_parser")


def find_consolidated_files(unzipped_dir: Path, pattern: str) -> List[Path]:
    """Retorna lista de arquivos que batem com o padrão (recursivo)."""
    return list(unzipped_dir.rglob(pattern))


def _ensure_category(df: pd.DataFrame, col: str) -> None:
    """Converte coluna para category quando apropriado (silencioso)."""
    try:
        if col in df.columns and not pd.api.types.is_categorical_dtype(df[col]):
            df[col] = df[col].astype("category")
    except Exception:
        pass


def _read_csv_safe(path: Path) -> Optional[pd.DataFrame]:
    """Lê um CSV com parâmetros padrão usados no pipeline; captura exceções e retorna None se falhar."""
    try:
        df = pd.read_csv(path, encoding="latin-1", sep=";", low_memory=False, dtype={"CNPJ_CIA": str})
        return df
    except Exception as exc:
        logger.warning("Falha lendo %s: %s", path, exc)
        return None


def _process_df_chunk(df: pd.DataFrame) -> Optional[pd.DataFrame]:
    """
    Aplicar filtros e tratamentos ao chunk (já lido).
    Retorna DataFrame filtrado pronto para concatenar ou None se vazio.
    """
    if "DT_FIM_EXERC" not in df.columns:
        return None

    for col in CATEGORY_COLS:
        if col in df.columns:
            _ensure_category(df, col)

    df["DT_FIM_EXERC"] = pd.to_datetime(df["DT_FIM_EXERC"], errors="coerce")
    df = df.dropna(subset=["DT_FIM_EXERC"])
    if df.empty:
        return None

    if "ORDEM_EXERC" in df.columns and "ESCALA_MOEDA" in df.columns:
        mask = (
            df["ORDEM_EXERC"].isin(["ÚLTIMO", "PENÚLTIMO"]) &
            df["ESCALA_MOEDA"].isin(["MIL", "UNIDADE"])
        )
        df = df.loc[mask].copy()

    if df.empty:
        return None

    if "VL_CONTA" in df.columns:
        df["VL_CONTA"] = pd.to_numeric(df["VL_CONTA"], errors="coerce")
        df = df.dropna(subset=["VL_CONTA"])
    else:
        return None

    if "ESCALA_MOEDA" in df.columns:
        if pd.api.types.is_categorical_dtype(df["ESCALA_MOEDA"]):
            cats = df["ESCALA_MOEDA"].cat.categories
            is_mil = df["ESCALA_MOEDA"].cat.codes == int(np.where(cats == "MIL")[0][0]) if "MIL" in cats else False
        else:
            is_mil = df["ESCALA_MOEDA"].astype(str) == "MIL"
        df["VL_CONTA"] = np.where(is_mil, df["VL_CONTA"] * 1000.0, df["VL_CONTA"])

    return df

def parse_and_consolidate_final(
    doc_name: str,
    broad_pattern: str,
    unzipped_dir: Path = UNZIPPED_DIR,
    processed_dir: Path = PROCESSED_DIR,
) -> Dict[str, object]:
    """
    Encontra arquivos consolidados (_con_), processa, concatena, deduplica e salva parquet/csv.
    Retorna dicionário com estatísticas do processamento.
    """
    logger.info("Iniciando processamento CONSOLIDADO para: %s (padrão: %s)", doc_name.upper(), broad_pattern)
    files = find_consolidated_files(unzipped_dir, broad_pattern)
    if not files:
        logger.info("Nenhum arquivo encontrado para o padrão: %s", broad_pattern)
        return {"status": "no_files", "files_count": 0}

    df_chunks = []
    total_rows_read = 0
    total_rows_after_filter = 0
    sapr_found = False

    for path in tqdm.tqdm(files, desc=f"Processando {doc_name.upper()}"):
        df = _read_csv_safe(path)
        if df is None:
            continue

        total_rows_read += len(df)

        if "CNPJ_CIA" in df.columns:
            df["CNPJ_CIA"] = df["CNPJ_CIA"].astype(str).str.strip()
            df_sapr = df[df["CNPJ_CIA"] == CNPJ_SAPR]
            if not df_sapr.empty and not sapr_found:
                sapr_found = True
                logger.info("[DIAGNÓSTICO SAPR11] Encontrado em: %s", path.name)
                if "ORDEM_EXERC" in df_sapr.columns:
                    logger.info("  ORDEM_EXERC values: %s", df_sapr["ORDEM_EXERC"].unique())
                if "ESCALA_MOEDA" in df_sapr.columns:
                    logger.info("  ESCALA_MOEDA values: %s", df_sapr["ESCALA_MOEDA"].unique())

        try:
            processed = _process_df_chunk(df)
            if processed is not None and not processed.empty:
                total_rows_after_filter += len(processed)
                df_chunks.append(processed)
        except Exception as exc:
            logger.warning("Erro processando arquivo %s: %s", path, exc)
        finally:
            del df
            gc.collect()

    logger.info("Totais: linhas lidas=%d, linhas após filtros=%d", total_rows_read, total_rows_after_filter)
    if not sapr_found:
        logger.warning("[DIAGNÓSTICO SAPR11] CNPJ Sanepar (%s) NÃO encontrado na busca consolidada.", CNPJ_SAPR)

    if not df_chunks:
        logger.info("Nenhum chunk com dados válidos após filtros. Abortando concatenação.")
        return {"status": "no_data_after_filter", "files_count": len(files)}

    logger.info("Concatenando %d chunks...", len(df_chunks))
    consolidated_df = pd.concat(df_chunks, ignore_index=True)
    consolidated_df.sort_values(by=["CNPJ_CIA", "DT_FIM_EXERC", "VERSAO"], ascending=[True, True, False], inplace=True)

    dedup_subset = ["CNPJ_CIA", "DT_FIM_EXERC", "CD_CONTA"]
    final_df = consolidated_df.drop_duplicates(subset=dedup_subset, keep="first").copy()

    if "CNPJ_CIA" in final_df.columns:
        debug_mask = final_df["CNPJ_CIA"].astype(str).isin([str(x) for x in CNPJs_DEBUG])
        debug_data = final_df.loc[debug_mask]
        if not debug_data.empty:
            grouped = debug_data.groupby(["CNPJ_CIA", "DT_FIM_EXERC"]).size().reset_index(name="contagem_contas")
            logger.info("Dados SAPR11/VIVA3 no DF final:\n%s", grouped.head(20).to_string(index=False))
        else:
            logger.info("Nenhum dado SAPR11/VIVA3 no DF final.")

    out_parquet = processed_dir / f"raw_{doc_name}.parquet"
    out_csv = processed_dir / f"raw_{doc_name}.csv"

    try:
        logger.info("Salvando Parquet: %s", out_parquet)
        final_df.to_parquet(out_parquet, index=False)
        logger.info("Salvando CSV: %s", out_csv)
        final_df.to_csv(out_csv, index=False, sep=";", encoding="utf-8-sig")
        logger.info("Shape final salvo: %s", final_df.shape)
    except Exception as exc:
        logger.exception("Falha ao salvar arquivos: %s", exc)
        return {"status": "save_error", "error": str(exc)}

    del df_chunks, consolidated_df, final_df
    gc.collect()

    return {
        "status": "ok",
        "files_count": len(files),
        "rows_read": total_rows_read,
        "rows_after_filter": total_rows_after_filter,
        "saved_parquet": str(out_parquet),
        "saved_csv": str(out_csv),
    }


if __name__ == "__main__":
    results = {}
    for name, pattern in DOC_PATTERNS_BROAD.items():
        results[name] = parse_and_consolidate_final(name, pattern)
        gc.collect()

    logger.info("Processo de parsing (v9.0 - Busca Consolidada) concluído!")
    logger.info("Consulte logs acima para mensagens '---> [DIAGNÓSTICO SAPR11 v9.0]'.")
    logger.info("Resumo por documento: %s", results)

2025-12-10 19:06:59,546 INFO Iniciando processamento CONSOLIDADO para: DRE (padrão: *_DRE_con_*.csv)
Processando DRE:   0%|          | 0/30 [00:00<?, ?it/s]C:\Users\kaike\AppData\Local\Temp\ipykernel_8772\438090661.py:47: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if col in df.columns and not pd.api.types.is_categorical_dtype(df[col]):
C:\Users\kaike\AppData\Local\Temp\ipykernel_8772\438090661.py:47: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if col in df.columns and not pd.api.types.is_categorical_dtype(df[col]):
C:\Users\kaike\AppData\Local\Temp\ipykernel_8772\438090661.py:47: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if col in df.columns and not pd.api.types.is_categori


### 3\. ⚙️ Script de Processamento: `03_cvm_processor.py`

### 📝 Descrição

Este é o cérebro da transformação. Ele pega os dados "LONG" (verticais), filtra apenas as contas contábeis que interessam para a análise financeira (Receita, Dívida, Caixa, EBIT) e realiza o **Pivot** para transformar em formato "WIDE" (horizontal). Por fim, une (Merge) as informações de Balanço (BPA/BPP) com Resultado (DRE).

  * **Funcionalidade Chave:** Mapeamento de contas principais e detalhadas (ex: separar "Caixa" de "Ativo Circulante").
  * **Resultado:** Um tabelão único pronto para cálculo de indicadores.

### 🔄 Fluxograma de Execução

```mermaid
graph LR
    A[Início] --> B[Carregar Parquets RAW]
    B --> C{Iterar Tipos<br>DRE, BPA, BPP}
    C --> D[Filtrar CD_CONTA<br>Principal + Detalhado]
    D --> E[Filtrar apenas<br>ÚLTIMO exercício]
    E --> F[Mapear CD_CONTA<br>para Nome Legível]
    F --> G[PIVOT TABLE<br>Long -> Wide]
    G --> H[Merge dos DataFrames<br>pela chave CNPJ + DATA]
    H --> I[Salvar fundamentals_wide.parquet]
```

### 💎 Dataset Final: `fundamentals_wide.parquet`

Este é o produto final do pipeline, pronto para ser consumido pelo seu `AurumQualityScoreCalculator`.

| Coluna | Descrição | Origem |
| :--- | :--- | :--- |
| **Identificadores** | | |
| `CNPJ_CIA` | Chave primária da empresa. | Todos |
| `DENOM_CIA` | Nome da empresa. | Todos |
| `DT_FIM_EXERC` | Data do balanço/resultado. | Todos |
| **Resultado (DRE)** | | |
| `Receita Líquida` | Faturamento líquido. | DRE 3.01 |
| `EBIT` | Lucro antes juros e impostos. | DRE 3.05 |
| `Lucro Líquido` | Lucro final consolidado. | DRE 3.11 |
| ... | *Outras contas de resultado* | ... |
| **Balanço (BPA)** | | |
| `Ativo Total` | Soma de todos os bens. | BPA 1 |
| `Ativo Circulante` | Bens de curto prazo. | BPA 1.01 |
| **`Caixa e Equivalentes`** | Dinheiro em caixa (Alta liquidez). | BPA 1.01.01 |
| **Passivo (BPP)** | | |
| `Passivo Total` | Soma das obrigações. | BPP 2 |
| `Patrimônio Líquido` | Capital dos sócios. | BPP 2.03 |
| **`Dívida Curto Prazo`** | Empréstimos vencendo em \< 1 ano. | BPP 2.01.04 |
| **`Dívida Longo Prazo`** | Empréstimos vencendo em \> 1 ano. | BPP 2.02.01 |

-----

### ✅ Conclusão

Com esta arquitetura, você transformou dados brutos, complexos e despadronizados da CVM em uma tabela analítica limpa (`fundamentals_wide`), contendo todas as variáveis necessárias para calcular métricas avançadas como **ROIC**, **Endividamento Líquido** e **Alavancagem**.

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import logging
from typing import Dict, List, Optional
import gc 

BASE_DIR = Path("..") / "data" / "cvm"
PROCESSED_DIR = BASE_DIR / "processed"
FINAL_DIR = BASE_DIR / "final"
FINAL_DIR.mkdir(parents=True, exist_ok=True)

MAPA_CONTAS_DRE_MAIN = {
    "3.01": "Receita Líquida",
    "3.02": "Custo dos Bens e/ou Serviços Vendidos",
    "3.03": "Lucro Bruto",
    "3.05": "EBIT",
    "3.07": "EBT",
    "3.11": "Lucro Líquido Consolidado",
}

MAPA_CONTAS_BPA_MAIN = {
    "1": "Ativo Total",
    "1.01": "Ativo Circulante",
    "1.02": "Ativo Não Circulante",
}

MAPA_CONTAS_BPP_MAIN = {
    "2": "Passivo Total",
    "2.01": "Passivo Circulante",
    "2.02": "Passivo Não Circulante",
    "2.03": "Patrimônio Líquido Consolidado",
}

MAPA_CONTAS_DETALHADAS_BPA = {
    "1.01.01": "Caixa e Equivalentes", 
}

MAPA_CONTAS_DETALHADAS_BPP = {
    "2.01.04": "Dívida Curto Prazo", 
    "2.02.01": "Dívida Longo Prazo", 
}

MAPA_CONTAS_GERAL = {
    "dre": {"main": MAPA_CONTAS_DRE_MAIN, "detailed": None},
    "bpa": {"main": MAPA_CONTAS_BPA_MAIN, "detailed": MAPA_CONTAS_DETALHADAS_BPA},
    "bpp": {"main": MAPA_CONTAS_BPP_MAIN, "detailed": MAPA_CONTAS_DETALHADAS_BPP},
}

INDEX_COLS = ["CNPJ_CIA", "DENOM_CIA", "DT_FIM_EXERC"]

logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s %(message)s")
logger = logging.getLogger("cvm_transform")

def _read_processed_parquet(path: Path) -> Optional[pd.DataFrame]:
    """Lê um parquet com tratamento de erro."""
    try:
        df = pd.read_parquet(path)
        logger.info("Lido parquet: %s (shape=%s)", path.name, df.shape)
        return df
    except Exception as exc:
        logger.warning("Falha ao ler parquet %s: %s", path.name, exc)
        return None

def process_and_pivot_file(
    doc_name: str,
    main_account_map: Dict[str, str],
    detailed_account_map: Optional[Dict[str, str]] = None,
    processed_dir: Path = PROCESSED_DIR,
) -> Optional[pd.DataFrame]:
    """
    Carrega raw parquet, filtra contas (principais E detalhadas),
    mantém 'ÚLTIMO' exercício e pivota para wide.
    """
    input_file = processed_dir / f"raw_{doc_name}.parquet"
    logger.info("Processando %s -> %s", doc_name.upper(), input_file.name)

    if not input_file.exists():
        logger.warning("Arquivo não encontrado: %s. Pulando %s.", input_file.name, doc_name)
        return None

    df = _read_processed_parquet(input_file)
    if df is None or df.empty: return None

    essential_cols = ["CD_CONTA", "VL_CONTA", "DT_FIM_EXERC", "ORDEM_EXERC"] + INDEX_COLS
    missing_essentials = [c for c in essential_cols if c not in df.columns]
    if missing_essentials:
        logger.error("Colunas essenciais ausentes em %s: %s. Pulando.", input_file.name, missing_essentials)
        return None

    all_accounts_to_keep = list(main_account_map.keys())
    if detailed_account_map:
        all_accounts_to_keep.extend(list(detailed_account_map.keys()))
        logger.info(f"Contas detalhadas a serem extraídas: {list(detailed_account_map.keys())}")

    logger.info(f"Total de códigos de conta a serem filtrados: {len(all_accounts_to_keep)}")
    df = df[df["CD_CONTA"].isin(all_accounts_to_keep)].copy()
    logger.info("Após filtrar contas (principais + detalhadas): shape=%s", df.shape)
    if df.empty:
        logger.info("Nenhuma conta de interesse encontrada. Pulando.")
        return None

    df["DT_FIM_EXERC"] = pd.to_datetime(df["DT_FIM_EXERC"], errors="coerce")
    df = df.dropna(subset=["DT_FIM_EXERC"])
    df = df[df["ORDEM_EXERC"] == "ÚLTIMO"].copy()
    logger.info("Após filtrar ORDEM_EXERC == 'ÚLTIMO': shape=%s", df.shape)
    if df.empty:
        logger.info("Nenhuma linha 'ÚLTIMO' encontrada. Pulando.")
        return None

    map_detalhado = detailed_account_map if detailed_account_map else {}
    df["CONTA"] = df["CD_CONTA"].map(map_detalhado).fillna(df["CD_CONTA"].map(main_account_map))
    df = df.dropna(subset=["CONTA"])
    logger.info("Contas mapeadas. Exemplo de nomes: %s", df["CONTA"].unique()[:5])
    if df.empty:
        logger.info("Nenhuma conta mapeada resultou em nome válido. Pulando.")
        return None

    df["VL_CONTA"] = pd.to_numeric(df["VL_CONTA"], errors="coerce")
    df = df.dropna(subset=["VL_CONTA"])
    if df.empty:
        logger.info("Sem valores numéricos para VL_CONTA. Pulando.")
        return None

    try:
        logger.info("Pivotando (long -> wide)...")
        df_wide = df.pivot_table(
            index=INDEX_COLS,
            columns="CONTA", 
            values="VL_CONTA",
            aggfunc="sum",
            fill_value=0 
        )
        df_wide = df_wide.reset_index()
        df_wide.columns.name = None 
        logger.info("Pivot concluído: shape=%s", df_wide.shape)
        logger.info("Colunas geradas pelo pivot: %s", df_wide.columns.tolist())
    except Exception as exc:
        logger.exception("Erro ao pivotar %s: %s", input_file.name, exc)
        return None

    del df
    gc.collect()
    return df_wide

def merge_fundamentals(dfs_wide: Dict[str, pd.DataFrame]) -> Optional[pd.DataFrame]:
    """ Junta os DataFrames wide (DRE, BPA, BPP) """
    if not dfs_wide:
        logger.warning("Nenhum DataFrame wide fornecido para merge.")
        return None
    valid_dfs = {k: v for k, v in dfs_wide.items() if v is not None and not v.empty}
    if not valid_dfs:
        logger.warning("Nenhum DataFrame wide VÁLIDO fornecido para merge.")
        return None

    keys = list(valid_dfs.keys())
    base = valid_dfs[keys[0]].copy()
    logger.info("Usando %s como base para merge (shape=%s)", keys[0], base.shape)

    for k in keys[1:]:
        logger.info("Mesclando com %s (shape=%s)", k, valid_dfs[k].shape)
        cols_to_merge = valid_dfs[k].columns.difference(base.columns).tolist() + INDEX_COLS
        base = pd.merge(base, valid_dfs[k][cols_to_merge], on=INDEX_COLS, how="outer")
        logger.info("Shape após merge com %s: %s", k, base.shape)

    base = base.sort_values(by=["CNPJ_CIA", "DT_FIM_EXERC"]).reset_index(drop=True)
    logger.info("Merge finalizado: shape=%s", base.shape)
    logger.info("Colunas finais: %s", base.columns.tolist())
    return base


def save_final(df: pd.DataFrame, final_dir: Path = FINAL_DIR, fname: str = "fundamentals_wide.parquet") -> Dict[str, str]:
    """ Salva o DataFrame final em parquet e CSV """
    final_dir.mkdir(parents=True, exist_ok=True)
    out_parquet = final_dir / fname
    out_csv = final_dir / str(fname).replace(".parquet", ".csv")

    try:
        logger.info("Salvando parquet final em: %s", out_parquet)
        df.to_parquet(out_parquet, index=False)
        logger.info("Salvando CSV final em: %s", out_csv)
        df.to_csv(out_csv, index=False, sep=";", encoding="utf-8-sig")
        return {"parquet": str(out_parquet), "csv": str(out_csv)}
    except Exception as exc:
        logger.exception("Erro ao salvar arquivo final: %s", exc)
        raise

if __name__ == "__main__":
    logger.info("Iniciando transformação para formato WIDE (com contas detalhadas)...")

    dfs_wide = {}
    for doc_type, maps_dict in MAPA_CONTAS_GERAL.items():
        main_map = maps_dict.get("main")
        detailed_map = maps_dict.get("detailed") 

        if main_map: 
            logger.info("-" * 20)
            df_wide = process_and_pivot_file(doc_type, main_map, detailed_map)
            if df_wide is not None and not df_wide.empty:
                dfs_wide[doc_type] = df_wide
            else:
                logger.warning("Processamento de %s não gerou DataFrame válido.", doc_type)
        else:
             logger.warning("Mapa principal não definido para %s. Pulando.", doc_type)

    if not dfs_wide:
        logger.error("Nenhum dataframe produzido. Encerrando sem salvar.")
    else:
        logger.info("-" * 20)
        logger.info("Iniciando merge dos DataFrames DRE, BPA, BPP...")
        final_df = merge_fundamentals(dfs_wide)
        if final_df is None or final_df.empty:
            logger.error("DataFrame final vazio após merge. Encerrando.")
        else:
            save_paths = save_final(final_df)
            logger.info("-" * 20)
            logger.info("Processamento concluído com sucesso!")
            logger.info("Arquivos salvos: %s", save_paths)
            logger.info("Shape final mestre: %s", final_df.shape)
            logger.info("Colunas finais geradas: %s", final_df.columns.tolist())
            logger.info("Amostra do resultado final:\n%s",
                        final_df.head()[INDEX_COLS + list(final_df.columns.difference(INDEX_COLS))].to_string(index=False))

2025-12-10 19:16:16,317 INFO Iniciando transformação para formato WIDE (com contas detalhadas)...
2025-12-10 19:16:16,318 INFO --------------------
2025-12-10 19:16:16,320 INFO Processando DRE -> raw_dre.parquet


2025-12-10 19:16:16,916 INFO Lido parquet: raw_dre.parquet (shape=(896850, 15))
2025-12-10 19:16:16,917 INFO Total de códigos de conta a serem filtrados: 6
2025-12-10 19:16:17,014 INFO Após filtrar contas (principais + detalhadas): shape=(150940, 15)
2025-12-10 19:16:17,095 INFO Após filtrar ORDEM_EXERC == 'ÚLTIMO': shape=(119831, 15)
2025-12-10 19:16:17,166 INFO Contas mapeadas. Exemplo de nomes: ['Receita Líquida' 'Custo dos Bens e/ou Serviços Vendidos' 'Lucro Bruto'
 'EBIT' 'EBT']
2025-12-10 19:16:17,183 INFO Pivotando (long -> wide)...
2025-12-10 19:16:17,324 INFO Pivot concluído: shape=(20041, 9)
2025-12-10 19:16:17,325 INFO Colunas geradas pelo pivot: ['CNPJ_CIA', 'DENOM_CIA', 'DT_FIM_EXERC', 'Custo dos Bens e/ou Serviços Vendidos', 'EBIT', 'EBT', 'Lucro Bruto', 'Lucro Líquido Consolidado', 'Receita Líquida']
2025-12-10 19:16:17,394 INFO --------------------
2025-12-10 19:16:17,395 INFO Processando BPA -> raw_bpa.parquet
2025-12-10 19:16:18,040 INFO Lido parquet: raw_bpa.parquet 